# LinkedIn Job scraper

## Imports

In [1]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from dotenv import load_dotenv

## Environment Variables

In [2]:
load_dotenv()
USERNAME=os.getenv('LINKEDIN_USERNAME')
PASSWORD=os.getenv('LINKEDIN_PASSWORD')
PROFILE_URL=os.getenv('LINKEDIN_PROFILE_URL')

print(USERNAME)

ericabraham806@gmail.com


## Login to LinkedIn

In [3]:
# Using headless browser
# options = webdriver.ChromeOptions()
# options.add_argument('headless')
# driver = webdriver.Chrome(options=options)

# Or

options = Options()
# options.add_argument('--headless=new')
options.add_argument('--window-size=1920,1200')
driver = webdriver.Chrome(options=options)

# driver = webdriver.Chrome()
driver.get('https://www.linkedin.com/login')
time.sleep(5)

username = driver.find_element(By.ID, 'username')
print(username)
username.send_keys(USERNAME)

password = driver.find_element(By.ID, 'password')
print(password)
password.send_keys(PASSWORD)

driver.find_element(By.XPATH, "//button[@type='submit']").click()

## Fetch Profile

In [6]:
url = PROFILE_URL
driver.get(url)

src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')

## Extract Profile Html

In [6]:
profile = soup.find('div', {'class': 'artdeco-hoverable-content__content'})
is_verified = 'has verifications' in profile.text
print(is_verified)

AttributeError: 'NoneType' object has no attribute 'text'

## Extract Jobs From Jobs Page

In [7]:
driver.find_element(By.XPATH, "//*[@id='global-nav']/div/nav/ul/li[3]/a/span").click()

In [8]:
src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')

## Extract Job Titles

In [ ]:
jobs = soup.find_all('a', {'class': 'job-card-list__title'})
titles = []
for title in jobs:
    titles.append(title.text.strip())
 
print(titles)

## Extract Companies

In [ ]:
companies = soup.find_all('span', {'class': 'job-card-container__primary-description'})
names = []
for name in companies:
    names.append(name.text.strip())
 
print(names)

## Extract Job Location

In [ ]:
import re
 
locations = soup.find_all('ul', {'class': 'job-card-container__metadata-wrapper'})
 
names = []
 
for location in locations:
    res = re.sub('\n\n +', ' ', location.text.strip())
    names.append(res)
 
print(names)

## Extract Jobs By Searching

In [ ]:
search = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
search.send_keys('software engineer')
search.send_keys(Keys.RETURN)

In [24]:
src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')

## Extract Job Title From Search

In [ ]:
jobs = soup.find_all('span', {'class': 'entity-result__title-text t-16'})
titles = []
for title in jobs:
    titles.append(title.text.strip())
 
print(titles)

In [34]:
search = driver.find_element(By.CLASS_NAME, "jobs-search-box__text-input")

In [ ]:
src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')

## Search Jobs in Jobs Page

In [9]:
keyword = driver.find_element(By.XPATH, "//input[contains(@id, 'jobs-search-box-keyword-id')]")
keyword.send_keys('software engineer')

In [10]:
location = driver.find_element(By.XPATH, "//input[contains(@id, 'jobs-search-box-location-id')]")
location.send_keys('Kigali')

In [11]:
location.send_keys(Keys.RETURN)

## Filter Jobs by [Jobs, Date, Remote, ...]

In [39]:
driver.find_element(By.XPATH, "//*[@id='search-reusables__filters-bar']/ul/li[1]/div/button").click()

## Organize Jobs in Table Form

In [12]:
element_class = 'jobs-search-results-list'
scrollable_element = driver.find_element("css selector", "." + element_class)
# total_height = scrollable_element.size["height"]
total_height = driver.execute_script("return arguments[0].scrollHeight;", scrollable_element)

print(total_height)

scroll_increment = 100
current_position = 0

while current_position < total_height:
    driver.execute_script("arguments[0].scrollTop += arguments[1];", scrollable_element, scroll_increment)
    current_position += scroll_increment
    time.sleep(0.5)

3399


In [13]:
html = driver.find_element(By.CLASS_NAME, "scaffold-layout__list-container")

driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [206]:
# element_class = 'jobs-search-results-list'

# scroll_script = """
# var element = document.getElementsByClassName('{0}')[0];
# element.scrollTop = element.scrollHeight;
# """.format(element_class)
# # driver.execute_script(scroll_script)
# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_element)

In [207]:
# element_class = 'jobs-search-results-list'
# scrollable_element = driver.find_element("css selector", "." + element_class)
# driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", scrollable_element)
# WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "." + element_class)))

[<selenium.webdriver.remote.webelement.WebElement (session="570c12022e58822fcbaeee35d1b8b86c", element="f.E4DCA506C6106D096858C5DF3BB1193A.d.39EC5DC0F0A6FB592900E73B4363CA18.e.257")>]

In [14]:
src = driver.page_source
soup = BeautifulSoup(src, 'html.parser')

In [15]:
import pandas as pd

In [16]:
titles = ['Job Title', 'Company', 'Location', 'Status', 'Application Link']

In [17]:
df = pd.DataFrame(columns = titles)

In [18]:
jobs = soup.find_all('li', {'class': 'jobs-search-results__list-item'})

for job in jobs:
    title = job.find('strong').text.strip()

    company = job.find('span', {'class': 'job-card-container__primary-description'}).text.strip()
    location = job.find('li', {'class': 'job-card-container__metadata-item'}).text.strip()
    status = job.find('div', {'class': 'job-card-container__job-insight-text'})
    status =  'N/A' if status is None else status.text.strip()
    link = job.find('a', href=True)['href']

    data = [job for job in (title, company, location, status, link)]

    length = len(df)
    df.loc[length] = data

df

,Job Title,Company,Location,Status,Application Link
0,"Software Engineer - Revit, C#, .NET",Braintrust,Sub-Saharan Africa (Remote),N/A,/jobs/view/3850035395/?eBP=CwEAAAGOYG05Zh3Sge7...
1,Senior Full Stack Web Developer,Braintrust,Sub-Saharan Africa (Remote),N/A,/jobs/view/3850037379/?eBP=CwEAAAGOYG05ZukeVta...
2,"Team Lead, Mobile Engineer (React/Capacitor/Ty...",Deel,EMEA (Remote),N/A,/jobs/view/3800416327/?eBP=CwEAAAGOYG05Z8CjoLM...
3,Senior Back-End Engineer (Node.js/TypeScript),Deel,EMEA (Remote),N/A,/jobs/view/3765008393/?eBP=CwEAAAGOYG05Z9ge3Et...
4,"Software Engineer, DeFi Metrics",Braintrust,EMEA (Remote),Your profile matches this job,/jobs/view/3812771508/?eBP=CwEAAAGOYG05ZyvrwDl...
5,Senior Backend Engineer,Right Balance ®,EMEA (Remote),N/A,/jobs/view/3860445166/?eBP=CwEAAAGOYG05ZzBfnsz...
6,Software Engineer Back-End,InstaDeep,"Kigali City, Rwanda (Hybrid)",N/A,/jobs/view/3851748357/?eBP=NOT_ELIGIBLE_FOR_CH...
7,Backend Engineer (Gateway),Carry1st,Sub-Saharan Africa (Remote),N/A,/jobs/view/3850809644/?eBP=NOT_ELIGIBLE_FOR_CH...
8,Full Stack Engineer,SiiRA World,EMEA (Remote),N/A,/jobs/view/3854416319/?eBP=NOT_ELIGIBLE_FOR_CH...
9,Back End Developer,Plexus Resource Solutions,EMEA (Remote),N/A,/jobs/view/3839792489/?eBP=NOT_ELIGIBLE_FOR_CH...


## Export to CSV

In [21]:
from datetime import datetime
df.to_csv(r'./Data/jobs_{}.csv'.format(datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')), index=False)